# Predicting the Stock Market

In this project, we'll work with the `sphist.csv` file containing index prices. 

Each row in the file contains a daily record of the price of the [S&P500 Index](https://en.wikipedia.org/wiki/S%26P_500_Index) from 1950 to 2015. 

The columns in the data-set are outlined below:

- `Date`: The date of the record.
- `Open`: The opening price of the day (when trading starts).
- `High`: The highest trade price during the day.
- `Low`: The lowest trade price during the day.
- `Close`: The closing price for the day (when trading is finished).
- `Volume`: The number of shares traded.
- `Adj Close`: The daily closing price, adjusted retroactively to include any corporate actions.

We'll be using this dataset to develop a predictive model. The model will be trained on data from 1950-2012, and then we'll try and use it to make predictions for 2013-2015. The `sphist.csv` can be found in the repository [here]().

To start with, we will import the libraries we need and then read in the `sphist.csv` file.

## Introduction

In [36]:
import pandas as pd 
import numpy as np
from datetime import datetime 

df = pd.read_csv('sphist.csv')

df.head(3)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117


We'll convert the `Date` column to a Pandas date type, using the [`pandas.to_datetime` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html).

In [37]:
df['Date'] = pd.to_datetime(df['Date'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
Date         16590 non-null datetime64[ns]
Open         16590 non-null float64
High         16590 non-null float64
Low          16590 non-null float64
Close        16590 non-null float64
Volume       16590 non-null float64
Adj Close    16590 non-null float64
dtypes: datetime64[ns](1), float64(6)
memory usage: 907.3 KB


Currently, the dataframe is sorted in descending order of date. We'll sort it in ascending order instead.

In [38]:
df_sorted = df.sort_values('Date', ascending=True)

df_sorted = df_sorted.reset_index(drop=True)

df_sorted.head(3)

,Date,Open,High,Low,Close,Volume,Adj Close
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93


## Generating new indicators

In a typical machine learning project, we might treat each row as independent. 

Stock market data however is *sequential*, and each observation comes a day after the previous observation. **Thus, the observations are not all independent, and we can't treat them as such.**

This means we have to be extra careful to not inject "future" knowledge into past rows when we do training and prediction. Injecting future knowledge would make our model look good when we're training and testing it, but would cause it to fail when applied in the real world. This is how many algorithmic traders lose money.

To account for this, **we'll instead generate 6 indicators for each row**.

The indicators will measure: 
- average price for the past 5 days
- average price for the past 30 days
- average price for the past 100 days
- standard deviation for price over the past 5 days
- standard deviation for price over the past 30 days
- standard deviation for price over the past 100 days

Each of these 6 indicators will be added on as columns to the `df_sorted` dataframe.

The [rolling function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html), which is a time series tool from pandas, will be used to perform most of the computation for us. The `windows` parameter will be used to specify the number of days in the past.

The `Close` column in the dataframe refers to the closing price for the day, so we'll use this to calculate the necessary indicators.

In [39]:
# Calculating the means:
mean_5 = df_sorted['Close'].rolling(window=5).mean().shift()
mean_30 = df_sorted['Close'].rolling(window=30).mean().shift()
mean_100 = df_sorted['Close'].rolling(window=100).mean().shift()

# Calculating the standard deviations:
std_5 = df_sorted['Close'].rolling(window=5).std().shift()
std_30 = df_sorted['Close'].rolling(window=30).std().shift()
std_100 = df_sorted['Close'].rolling(window=100).std().shift()

Since the rolling mean/std used the current day's price, we had to reindex the resulting series to shift all the values "forward" one day. 

For example, the rolling mean calculated for 1950-01-03 needed to be assigned to 1950-01-04, etc. We did this using the [shift](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html) method above.

Next, we'll add these indicators into the `df_sorted` dataframe.

In [40]:
df_sorted['mean_5'] = mean_5
df_sorted['mean_30'] = mean_30
df_sorted['mean_100'] = mean_100
df_sorted['std_5'] = std_5
df_sorted['std_30'] = std_30
df_sorted['std_100'] = std_100

In [41]:
df_sorted.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_100,std_5,std_30,std_100
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN,NaN
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,NaN,NaN,0.157956,NaN,NaN
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,NaN,NaN,0.089051,NaN,NaN
7,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,NaN,NaN,0.067602,NaN,NaN
8,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,NaN,NaN,0.134796,NaN,NaN
9,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,NaN,NaN,0.196545,NaN,NaN


We see that there are some `NaN` values, where there weren't enough previous days to calculate the value for that column. We'll drop these rows.

In [42]:
df_final = df_sorted.dropna()

df_final.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_100,std_5,std_30,std_100
100,1950-05-26,18.670000,18.670000,18.670000,18.670000,1330000.0,18.670000,18.674,18.214667,17.5128,0.042778,0.290454,0.549764
101,1950-05-29,18.719999,18.719999,18.719999,18.719999,1110000.0,18.719999,18.672,18.238333,17.5329,0.042661,0.297821,0.554989
102,1950-05-31,18.780001,18.780001,18.780001,18.780001,1530000.0,18.780001,18.696,18.266333,17.5516,0.019493,0.302421,0.563190
103,1950-06-01,18.770000,18.770000,18.770000,18.770000,1580000.0,18.770000,18.710,18.291333,17.5701,0.043012,0.313025,0.572867
104,1950-06-02,18.790001,18.790001,18.790001,18.790001,1450000.0,18.790001,18.726,18.315333,17.5880,0.048270,0.321374,0.582133
105,1950-06-05,18.600000,18.600000,18.600000,18.600000,1630000.0,18.600000,18.746,18.344000,17.6051,0.050300,0.324161,0.592090
106,1950-06-06,18.879999,18.879999,18.879999,18.879999,2250000.0,18.879999,18.732,18.365333,17.6208,0.078550,0.319037,0.597477
107,1950-06-07,18.930000,18.930000,18.930000,18.930000,1750000.0,18.930000,18.764,18.400333,17.6387,0.101636,0.315862,0.608133
108,1950-06-08,19.139999,19.139999,19.139999,19.139999,1780000.0,19.139999,18.794,18.437000,17.6604,0.126609,0.311184,0.615137
109,1950-06-09,19.260000,19.260000,19.260000,19.260000,2130000.0,19.260000,18.868,18.483000,17.6851,0.197408,0.309651,0.624486


## Creating train and test dataframes

Now, we'll generate two new dataframes to use in making our algorithm. 

The `train` dataframe will contain any rows in the data with a date before 2013-01-01. The `test` dataframe will contain any rows with a date equal or after 2013-01-01.

In [44]:
train = df_final[df_final['Date'] < datetime(year=2013, month=1, day=1)]
test = df_final[df_final['Date'] >= datetime(year=2013, month=1, day=1)]

In [47]:
train.tail(3) # Seeing last 3 dates in train

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_100,std_5,std_30,std_100
15848,2012-12-27,1419.829956,1422.800049,1401.800049,1418.099976,2.830180e+09,1418.099976,1431.228003,1408.813000,1420.068599,9.058684,24.055972,25.403941
15849,2012-12-28,1418.099976,1418.099976,1401.579956,1402.430054,2.426680e+09,1402.430054,1427.685986,1410.265332,1420.599598,10.208568,23.215373,24.788763
15850,2012-12-31,1402.430054,1426.739990,1398.109985,1426.189941,3.204330e+09,1426.189941,1419.434009,1411.830001,1420.713999,10.701861,20.858522,24.676879


In [46]:
test.head(3) # Seeing first 3 dates in train

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_100,std_5,std_30,std_100
15851,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1414.258667,1421.033599,9.820801,17.834740,24.536976
15852,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1417.676668,1421.644299,22.261321,16.852563,24.800691
15853,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1420.092668,1422.215800,26.274326,17.470824,25.006184


## Using Linear Regression

We'll now perform Linear Regression to test the accuracy of our model.

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [60]:
# Using List Comprehension to generate a list of the feature columns.
features = [x for x in train.columns if x.startswith("mean") or x.startswith("std")]

# Instantiating lr 
lr = LinearRegression()

# Training Model.
lr.fit(train[features], train['Close'])

# Predicting on test set
predictions = lr.predict(test[features])

# Calculating Error
rmse = mean_squared_error(predictions, test['Close']) ** 0.5

print("The root mean squared error is:", rmse)

The root mean squared error is: 22.202832665588158


In [61]:
print(lr.score(train[features], train['Close']))

0.99952899848177


We saw that the RMSE of our model is around 22.2, which is a reasonably low error. 

Some other metrics that might be useful to test include: 
- The ratios between standard deviation for the past 5 days, and standard deviation for the past 30 days.
- The ratios between mean for the past 5 days, and mean for the past 30 days. 
- The day of the Week. 
- The number of Holidays in the prior month. 

We'll add the first two of those indicators and see if it reduces the error of our model. 

## Adding 2 new indicators

We'll once again return to the `df_sorted` dataframe, and add two new columns:
1. Ratio between std for past 5 days, and std for past 30 days.
2. Ratio between mean for past 5 days, and mean for past 30 days.

In [63]:
df_sorted.head(6)

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_100,std_5,std_30,std_100
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN,NaN
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.9,NaN,NaN,0.157956,NaN,NaN


In [65]:
df_sorted['mean_ratio'] = df_sorted['mean_30'] / df_sorted['mean_5']
df_sorted['std_ratio'] = df_sorted['std_30'] / df_sorted['std_5']

df_final2 = df_sorted.dropna() # Dropping NaN rows

df_final2.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_100,std_5,std_30,std_100,mean_ratio,std_ratio
100,1950-05-26,18.670000,18.670000,18.670000,18.670000,1330000.0,18.670000,18.674,18.214667,17.5128,0.042778,0.290454,0.549764,0.975403,6.789732
101,1950-05-29,18.719999,18.719999,18.719999,18.719999,1110000.0,18.719999,18.672,18.238333,17.5329,0.042661,0.297821,0.554989,0.976774,6.981027
102,1950-05-31,18.780001,18.780001,18.780001,18.780001,1530000.0,18.780001,18.696,18.266333,17.5516,0.019493,0.302421,0.563190,0.977018,15.514383
103,1950-06-01,18.770000,18.770000,18.770000,18.770000,1580000.0,18.770000,18.710,18.291333,17.5701,0.043012,0.313025,0.572867,0.977623,7.277655
104,1950-06-02,18.790001,18.790001,18.790001,18.790001,1450000.0,18.790001,18.726,18.315333,17.5880,0.048270,0.321374,0.582133,0.978070,6.657804
105,1950-06-05,18.600000,18.600000,18.600000,18.600000,1630000.0,18.600000,18.746,18.344000,17.6051,0.050300,0.324161,0.592090,0.978555,6.444592
106,1950-06-06,18.879999,18.879999,18.879999,18.879999,2250000.0,18.879999,18.732,18.365333,17.6208,0.078550,0.319037,0.597477,0.980426,4.061588
107,1950-06-07,18.930000,18.930000,18.930000,18.930000,1750000.0,18.930000,18.764,18.400333,17.6387,0.101636,0.315862,0.608133,0.980619,3.107764
108,1950-06-08,19.139999,19.139999,19.139999,19.139999,1780000.0,19.139999,18.794,18.437000,17.6604,0.126609,0.311184,0.615137,0.981005,2.457826
109,1950-06-09,19.260000,19.260000,19.260000,19.260000,2130000.0,19.260000,18.868,18.483000,17.6851,0.197408,0.309651,0.624486,0.979595,1.568586


### Running Linear Regression Again

Once more, we'll split the dataframe into the train and test sets.

In [66]:
train2 = df_final2[df_final2['Date'] < datetime(year=2013, month=1, day=1)]
test2 = df_final2[df_final2['Date'] >= datetime(year=2013, month=1, day=1)]

In [67]:
train2.tail(3)

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_100,std_5,std_30,std_100,mean_ratio,std_ratio
15848,2012-12-27,1419.829956,1422.800049,1401.800049,1418.099976,2.830180e+09,1418.099976,1431.228003,1408.813000,1420.068599,9.058684,24.055972,25.403941,0.984339,2.655570
15849,2012-12-28,1418.099976,1418.099976,1401.579956,1402.430054,2.426680e+09,1402.430054,1427.685986,1410.265332,1420.599598,10.208568,23.215373,24.788763,0.987798,2.274107
15850,2012-12-31,1402.430054,1426.739990,1398.109985,1426.189941,3.204330e+09,1426.189941,1419.434009,1411.830001,1420.713999,10.701861,20.858522,24.676879,0.994643,1.949056


In [68]:
train2.head(3)

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_100,std_5,std_30,std_100,mean_ratio,std_ratio
100,1950-05-26,18.670000,18.670000,18.670000,18.670000,1330000.0,18.670000,18.674,18.214667,17.5128,0.042778,0.290454,0.549764,0.975403,6.789732
101,1950-05-29,18.719999,18.719999,18.719999,18.719999,1110000.0,18.719999,18.672,18.238333,17.5329,0.042661,0.297821,0.554989,0.976774,6.981027
102,1950-05-31,18.780001,18.780001,18.780001,18.780001,1530000.0,18.780001,18.696,18.266333,17.5516,0.019493,0.302421,0.563190,0.977018,15.514383


Next, we'll run Linear Regression again.

In [71]:
# Using List Comprehension to generate a list of the feature columns.
features = [x for x in train2.columns if x.startswith("mean") or x.startswith("std")]

# Instantiating lr 
lr = LinearRegression()

# Training Model.
lr.fit(train2[features], train2['Close'])

# Predicting on test set
predictions = lr.predict(test2[features])

# Calculating Error
rmse = mean_squared_error(predictions, test2['Close']) ** 0.5

print("The root mean squared error is:", rmse)

The root mean squared error is: 22.180001289989615


In [74]:
print(lr.score(train2[features], train2['Close']))

0.9995294357128204


We see that we managed to improve accuracy of the model marginally, by lowering the RMSE from around 22.20, to 22.18.

## Future Improvements to accuracy of model

There is definitely a lot of room for improvement to the model's predictive accuracy.

Some such improvements could be:

1. Including even more indicators, like day of the week, no. of holidays in the previous month, etc. 
2. Making Predictions only up to a week, or even only a day in advance. 
    - For instance, data from 2014-05-20 and earlier could be used to predict prices on 2014-05-21.
    - This more closely resembles how people actually do stock market prediction using algorithms.
3. Using different algorithms, like a random forest, and seeing if they perform better.
4. Inclusion of data external to the provided dataset. For instance, we could look at things like:
    - The weather in cities where most trading happens
    - The amount of twitter activity surrounding a certain stock
    - Sentiment Analysis of a certain stock, using for instance Twitter data.
5. Making the system real-time by writing an automated script to download the latest data when the market closes, and then using that to make predictions for the next day.